# 4. Télémétrie AKS

<img src='https://github.com/retkowsky/images/blob/master/AzureMLservicebanniere.png?raw=true'>

## 1. Informations

In [1]:
import sys
sys.version

'3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) \n[GCC 7.3.0]'

In [2]:
import datetime
now = datetime.datetime.now()
print(now)

2020-03-16 13:10:38.190370


In [3]:
import azureml.core
print("Version Azure ML service :", azureml.core.VERSION)

Version Azure ML service : 1.0.83


In [4]:
import azureml.core
from azureml.core import Workspace
from azureml.core.webservice import Webservice
from azureml.core.authentication import AzureCliAuthentication
import requests
import json

In [5]:
#Azure subscription
subscription_id = "70b8f39e-8863-49f7-b6ba-34a80799550c" 

#Azure resource group 
resource_group = "MLOpsworkshopRG" 

#Azure ML workspace et région
workspace_name = "MLOpsWorkshop" 
workspace_region = "westeurope" 

# Infos endpoint
rest_service_name = "chd-predictor-service"
rest_service_url = "http://13.77.72.52:80/api/v1/service/chd-predictor-service/score" 
rest_service_key = "1Wj4hBydO5ebD2sQFf1m02LSgGh0BeJA" 

In [6]:
ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region,
    exist_ok = True)

ws.write_config()
print('Workspace configuration succeeded')

Workspace configuration succeeded


## 2. Call du modèle

In [20]:
# Pour tester
test_data = {"data":[[61, 1, 150],[67, 2, 300]]}

In [21]:
test_data

{'data': [[61, 1, 150], [67, 2, 300]]}

In [23]:
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ rest_service_key)}
response = requests.post(rest_service_url, json.dumps(test_data), headers=headers)
print('Prédictions :')
print(response.text)

Prédictions :
"[0, 1]"


## 3. Activation télémétrie

In [24]:
rest_service = Webservice(ws, rest_service_name)

In [25]:
rest_service.update(enable_app_insights=True, collect_model_data=True)

## 4. Génération d'appels pour simuler des données de télémétries

In [28]:
test_data = {"data":[[44, 1, 138],[43, 1, 150],[67,1,138]]}

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ rest_service_key)}
response = requests.post(rest_service_url, json.dumps(test_data), headers=headers)
print('Prédictions :')
print(response.text)

Prédictions :
"[0, 0, 1]"


In [29]:
test_data = {"data":[[61, 1, 150]]}

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ rest_service_key)}
response = requests.post(rest_service_url, json.dumps(test_data), headers=headers)
print('Prédiction :')
print(response.text)

Prédiction :
"[0]"


In [30]:
test_data = {"data":[[43, 1, 180]]}

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ rest_service_key)}
response = requests.post(rest_service_url, json.dumps(test_data), headers=headers)
print('Prédiction :')
print(response.text)

Prédiction :
"[0]"


In [31]:
test_data = {"data":[[67,1,138]]}

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ rest_service_key)}
response = requests.post(rest_service_url, json.dumps(test_data), headers=headers)
print('Prédiction :')
print(response.text)

Prédiction :
"[1]"


In [32]:
test_data = {"data":[[68,1,189]]}

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ rest_service_key)}
response = requests.post(rest_service_url, json.dumps(test_data), headers=headers)
print('Predictions :')
print(response.text)

Predictions :
"[1]"


Aller dans le portail Azure pour visualiser les informations de télémétries disponibles depuis AKS

## 5. Infos service

In [38]:
print("Etat : ", rest_service.state)

Etat :  Healthy


In [39]:
print("Endpoint : ", rest_service.scoring_uri)

Endpoint :  http://13.77.72.52:80/api/v1/service/chd-predictor-service/score


In [36]:
# GetLogs
rest_service.get_logs()

'2020-03-12T13:34:44,500905616+00:00 - gunicorn/run \n2020-03-12T13:34:44,500042511+00:00 - iot-server/run \n2020-03-12T13:34:44,501077817+00:00 - rsyslog/run \n2020-03-12T13:34:44,503254627+00:00 - nginx/run \nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2020-03-12T13:34:44,577354497+00:00 - iot-server/finish 1 0\n2020-03-12T13:34:44,578465503+00:00 - Exit code 1 is normal. Not restarting iot-server.\nStarting gunicorn 19.6.0\nListening at: http://127.0.0.1:31311 (10)\nUsing worker: sync\nworker timeout is set to 300\nBooting worker with pid: 47\nInitializing logger\nStarting up app insights client\nStarting up request id generator\nStarting up app insight hooks\nInvoking user\'s init function\nLooking for model path for model:\nchd-predictor\nLoading model from:\nazureml-models/chd-predictor/2/chd-rf-model\nModel loaded from disk.\n\'RandomForestClassifier\' object has no attribute \'summary\'\nUsers\'s init has completed successfully\nSkipping middlewa

In [ ]:
#rest_service.delete()

> Fin